Importing Libraries

In [1]:
!pip install -qU langsmith langchain-core langchain-community langchain-openai langchain-qdrant
!pip install -qU pymupdf ragas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.64 requires langchain-community<0.3.0,>=0.2.10, but you have langchain-community 0.3.0 which is incompatible.
langchain-experimental 0.0.64 requires langchain-core<0.3.0,>=0.2.27, but you have langchain-core 0.3.2 which is incompatible.
langgraph 0.2.16 requires langchain-core<0.3,>=0.2.27, but you have langchain-core 0.3.2 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.2 which is incompatible.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.2 which is incompatible.
langgraph-checkpoint 1.0.6 requires langchain-core<0.3,>=0.2.22, but you have langchain-core 0.3.2 which is incompatible.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document

In [3]:
# set up openapi key to .env.local File

from dotenv import load_dotenv
import os

load_dotenv(".env.local")
api_key = os.getenv("OPENAI_API_KEY")

# Validate the API key
if not api_key or api_key == "your-api*****here":
    raise ValueError("Invalid API key. Please set a valid OPENAI_API_KEY in your .env.local file.")

print(f"API key loaded and validated.")

API key loaded and validated.


In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env.local
load_dotenv(".env.local")

# Set LANGCHAIN_TRACING_V2
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Get LANGCHAIN_API_KEY from .env.local
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

if not langchain_api_key:
    raise ValueError("LANGCHAIN_API_KEY not found in .env.local file")

os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

print("LangChain API Key loaded from .env.local")

LangChain API Key loaded from .env.local


TASK 1: Dealing with the Data

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

# Load the first document
documents1 = PyMuPDFLoader(file_path="https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf").load()

# Load the second document
documents2 = PyMuPDFLoader(file_path="https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf").load()

Chunking the Data and Splitting

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    
)

documents1 = text_splitter.split_documents(documents1)
documents2 = text_splitter.split_documents(documents2)


In [7]:
len(documents1)

1752

In [8]:
len(documents2)

1146

Text Embedding Model to create Vector Store

In [9]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-3-small"

embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    openai_api_key=api_key
)

Setting up Qdrant Vector Store

In [10]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "ai_ethics_docs"
VECTOR_SIZE = 1536

Loading the Data into Qdrant Vector Store

In [11]:
qdrant_client = QdrantClient(
    location=LOCATION
)

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings
)

qdrant_vector_store.add_documents(documents1)
qdrant_vector_store.add_documents(documents2)

['5e813793b47047ec9b590734beb9b562',
 '6affeecb5dad416d949ab48b22eb93a4',
 '96be3bc3ff444f2888622f7baeb181ac',
 '8ce426c9cd4140328ade5d365c2f8a14',
 'fe6e0a1479bb4d78b58f267486d22cff',
 '9fd9990014a346b6b40d45844dccd198',
 'fb189d83de2c4295b340bbb1bc4c88c2',
 'af09afea5ca64e87bb860523cf9cefe2',
 '5ecaa1423e664201a81fee72b00d3671',
 '5e5e52d046b54baa8ab4e80562234bfa',
 '237f31fea4e24c1ab34d52e36274005b',
 '5fc9d99e20a0404dbe618942fc48568d',
 '8c87cd3c034f426b8ed00f3b5f933112',
 'f3d23137381f493ba0fa3d0c8e1b68b9',
 'c32f64789a564aa7b1a2362e7b7b4dd9',
 '2bc3d54f456747a5b1d92aa8539ed00d',
 '26a1e6371bb0457183974521ea412877',
 '372a20241a564699a9f764f06aceeaef',
 '318be871f6b64bb49eeb0e991ea694aa',
 '87350fe8d4434b6dac9c689d41cf2c9f',
 'f5e8c512dae74645943e6c8d890d0d4d',
 '6d8597a7b7524f95a480371338b4458a',
 '3d66e1e6187f40de9547a3d493cee831',
 '70405eeaeb16463fb484a5700e7f5af8',
 'c3594019322f46b9839fe07413b3713f',
 'df2aa1e6a5b7420b89bc400a52ff8e8d',
 '0f4266f94dfd4e84b6651c478ed01167',
 

RAG Prototype

Setting up the Retriever

In [12]:
retriever = qdrant_vector_store.as_retriever()

retrieved_documents = retriever.invoke("How can I protect my privacy?")

for doc in retrieved_documents:
  print(doc)

page_content='Protect privacy by design and by default 
Privacy by design and by default. Automated systems should be designed and built with privacy protect­' metadata={'source': 'https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf', 'file_path': 'https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf', 'page': 32, 'total_pages': 73, 'format': 'PDF 1.6', 'title': 'Blueprint for an AI Bill of Rights', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe Illustrator 26.3 (Macintosh)', 'producer': 'iLovePDF', 'creationDate': "D:20220920133035-04'00'", 'modDate': "D:20221003104118-04'00'", 'trapped': '', '_id': '69fcbf9896f6412ba371ba72974ec0a4', '_collection_name': 'ai_ethics_docs'}
page_content='You should be protected from abusive data practices via built-in 
protections and you should have agency over how data about 
you is used. You should be protected from violations of privacy through' metadata

Creating the RAG Chain

In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

print(retrieval_qa_prompt.messages[0].prompt.template)


Answer any use questions based solely on the context below:

<context>
{context}
</context>


In [16]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a helpful assistant. Act as an AI ethics expert and answer the question in a succinct way. 
If you cannot answer the question based on the context - you must say "I don't know".

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

In [39]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {
        "response": prompt | primary_qa_llm, 
        "context": itemgetter("context")
    }
)

In [40]:
# Example query
query = "How do I protect my privacy?"
response = retrieval_augmented_qa_chain.invoke({"question": query})
print(response["response"].content)

To protect your privacy, ensure that automated systems are designed with built-in privacy protections, data minimization, limitations on data use and collection, and transparency. You should also have agency over how your data is used and be protected from abusive data practices.


In [41]:
#Example querty 2

query = "What is a rule of thumb for selecting an industry to invest in?"
response = retrieval_augmented_qa_chain.invoke({"question": query})
print(response["response"].content)

I don't know.


Deploying the RAG Prototype to Hugging Face


Creating Test Data

In [25]:
eval_documents1 = PyMuPDFLoader(file_path="https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf").load()
eval_documents2 = PyMuPDFLoader(file_path="https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf").load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents1 = text_splitter_eval.split_documents(eval_documents1)
eval_documents2 = text_splitter_eval.split_documents(eval_documents2)

In [26]:
len(eval_documents1)

439

In [27]:
len(eval_documents2)

322

In [31]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 10 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset1 = generator.generate_with_langchain_docs(eval_documents1, num_qa_pairs, distributions)
testset2 = generator.generate_with_langchain_docs(eval_documents2, num_qa_pairs, distributions)

testset1.to_pandas()
testset2.to_pandas()

embedding nodes:   0%|          | 0/878 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

embedding nodes:   0%|          | 0/644 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How does the GAI system interact with external...,[other systems. \nInformation Integrity; Value...,Observe and analyze how the GAI system interac...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
1,How are factually accurate outputs produced by...,[way generative models are designed: they gene...,Generative models produce factually accurate o...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
2,What is the significance of Algorithmic Discri...,[when those patients had similar numbers of ch...,Algorithmic Discrimination Protections are sig...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
3,How can methods to trace the origin and modifi...,[implementation starting with the most signiﬁc...,Methods to trace the origin and modifications ...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
4,"How can organizations document trade-offs, dec...",[40 \nMANAGE 1.3: Responses to the AI risks de...,Responses to the AI risks deemed high priority...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
5,How to align incident response plans for third...,[22 \nGV-6.2-003 \nEstablish incident response...,Aligning incident response plans for third-par...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
6,How do incident reports help identify harms in...,"[uses of AI systems in similar contexts, publi...",Incident reports help identify harms in GAI sy...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
7,How do AI systems affect equal opportunities i...,[The individual panelists described the ways i...,AI systems and other technologies are increasi...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
8,How to measure bias in GAI system outputs for ...,[36 \nMEASURE 2.11: Fairness and bias – as ide...,Bias in GAI system outputs for different demog...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
9,What framework is NIST developing for AI risk ...,[points to numerous examples of effective and ...,The NIST is developing an AI Risk Management F...,reasoning,[{'source': 'https://www.whitehouse.gov/wp-con...,True


In [32]:
testset1.test_data[0]
testset2.test_data[0]

DataRow(question='How does the GAI system interact with external networks and what potential negative externalities can arise, particularly in relation to compromised content provenance?', contexts=['other systems. \nInformation Integrity; Value Chain \nand Component Integration \nMP-2.2-002 \nObserve and analyze how the GAI system interacts with external networks, and \nidentify any potential for negative externalities, particularly where content \nprovenance might be compromised. \nInformation Integrity \nAI Actor Tasks: End Users \n \nMAP 2.3: Scientiﬁc integrity and TEVV considerations are identiﬁed and documented, including those related to experimental \ndesign, data collection and selection (e.g., availability, representativeness, suitability), system trustworthiness, and construct'], ground_truth='Observe and analyze how the GAI system interacts with external networks, and identify any potential for negative externalities, particularly where content provenance might be compromi

In [33]:
testset_df1 = testset1.to_pandas()
testset_df1.to_csv("synthetic_data1.csv")

testset_df2 = testset2.to_pandas()
testset_df2.to_csv("synthetic_data2.csv")

In [34]:
import pandas as pd

test_df = pd.read_csv("synthetic_data1.csv")
test_df2 = pd.read_csv("synthetic_data2.csv")


In [35]:
testset_df1

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What requirements must be met before a localit...,[individuals protected from discrimination by ...,Before a locality can use a pretrial risk asse...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
1,Who is the Deputy Chief Technology Officer for...,"[Technology Policy\n•\nJ. Bob Alotta, Vice Pre...",Kathy Pham Evans is the Deputy Chief Technolog...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
2,How should individual privacy be balanced with...,"[oversee. In the case of public sector uses, t...",Care should be taken to balance individual pri...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
3,How does automated hiring screening contribute...,[housing complexes. Employment-related concern...,Automated hiring screening contributes to conc...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
4,How should organizations approach narrowing th...,"[narrow identified goals, to avoid ""mission cr...",Organizations should narrow their identified g...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
5,What are transgender travelers saying about ai...,[scanning setting based on the passenger’s sex...,Transgender travelers have described degrading...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
6,How can the system ensure fair access for unde...,"[HUMAN ALTERNATIVES, \nCONSIDERATION, AND \nFA...",The system can ensure fair access for underser...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
7,How to ensure accessibility for users with dis...,"[otherwise provided, should be easy to find an...",Mechanisms for human consideration and fallbac...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
8,What steps are needed for hardware maintenance...,"[any hardware components, data inputs, etc.) a...",The answer to given question is not present in...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
9,How does the AI Bill of Rights benefit America...,[Management and Budget (OMB) with an annual re...,The answer to given question is not present in...,reasoning,[{'source': 'https://www.whitehouse.gov/wp-con...,True


In [36]:
test_df2

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,How does the GAI system interact with external...,['other systems. \nInformation Integrity; Valu...,Observe and analyze how the GAI system interac...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
1,1,How are factually accurate outputs produced by...,['way generative models are designed: they gen...,Generative models produce factually accurate o...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
2,2,What is the significance of Algorithmic Discri...,['when those patients had similar numbers of c...,Algorithmic Discrimination Protections are sig...,simple,[{'source': 'https://www.whitehouse.gov/wp-con...,True
3,3,How can methods to trace the origin and modifi...,['implementation starting with the most signiﬁ...,Methods to trace the origin and modifications ...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
4,4,"How can organizations document trade-offs, dec...",['40 \nMANAGE 1.3: Responses to the AI risks d...,Responses to the AI risks deemed high priority...,simple,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
5,5,How to align incident response plans for third...,['22 \nGV-6.2-003 \nEstablish incident respons...,Aligning incident response plans for third-par...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
6,6,How do incident reports help identify harms in...,"[""uses of AI systems in similar contexts, publ...",Incident reports help identify harms in GAI sy...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
7,7,How do AI systems affect equal opportunities i...,['The individual panelists described the ways ...,AI systems and other technologies are increasi...,multi_context,[{'source': 'https://www.whitehouse.gov/wp-con...,True
8,8,How to measure bias in GAI system outputs for ...,['36 \nMEASURE 2.11: Fairness and bias – as id...,Bias in GAI system outputs for different demog...,multi_context,[{'source': 'https://nvlpubs.nist.gov/nistpubs...,True
9,9,What framework is NIST developing for AI risk ...,['points to numerous examples of effective and...,The NIST is developing an AI Risk Management F...,reasoning,[{'source': 'https://www.whitehouse.gov/wp-con...,True


Fine-Tuning the Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

def fine_tune_embeddings(train_data, model_name="sentence-transformers/all-MiniLM-L6-v2", epochs=3):
    model = SentenceTransformer(model_name)
    
    train_examples = [InputExample(texts=[item['question'], item['answer']]) for item in train_data]
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    
    train_loss = losses.MultipleNegativesRankingLoss(model)
    
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=epochs, warmup_steps=100)
    
    return model

# Usage
fine_tuned_model = fine_tune_embeddings(train_data)

Assessing Performance

In [ ]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

def assess_performance(rag_chain, test_data):
    results = evaluate(
        rag_chain,
        test_data,
        metrics=[
            faithfulness(),
            answer_relevancy(),
            context_precision(),
            context_recall()
        ]
    )
    return results

# Usage
performance_results = assess_performance(rag_chain, test_data)
print(performance_results)